In [1]:
import numpy as np
import pickle
%matplotlib inline
import matplotlib.pyplot as plt
import sys, os
sys.path.append(os.environ['rapp'])
sys.path.append(os.environ['raco'])
sys.path.append(os.environ['rapl'] + '/azav')
from azav_util import plot_azav
from common import *
from plotcommon import *
from constants import *
labelsize = 8

In [3]:
dirname = '/nobackup/lmatilsk/case_M_tspec-lowcad/'
datadir = dirname + 'data/'
mtracedir = datadir + 'mtrace_mmax010/'

In [10]:
radlevs = get_slice_levels(dirname)
print ('irvals     =', radlevs.inds)
print ('rvals      = ' + arr_to_str(radlevs.radius, "%1.3e"))
print ('rvals/rsun = ' + arr_to_str(radlevs.radius/rsun, "%.3f"))

irvals     = [ 73  85  99 137 147 155 161 168 176 182]
rvals      = [4.985e+10 4.919e+10 4.805e+10 4.599e+10 4.414e+10 4.198e+10 4.018e+10 3.810e+10 3.606e+10 3.494e+10]
rvals/rsun = [0.717 0.707 0.691 0.661 0.634 0.603 0.578 0.548 0.518 0.502]


In [23]:
# Get necessary grid info
di_grid = get_grid_info(dirname)
rr = di_grid['rr']
cost = di_grid['cost']
sint = di_grid['sint']
tt_lat = di_grid['tt_lat']
xx = di_grid['xx']
tt = di_grid['tt']
nt = di_grid['nt']
nr = di_grid['nr']

In [14]:
irval = 2 
part = 'real'
mval = 1

if part == 'imag':
    partfunc = np.imag
elif part == 'abs':
    partfunc = np.abs
elif part == 'real':
    partfunc = np.real

# read in induction data
qval = 802
the_file = get_widest_range_file(mtracedir, 'mtrace_qval%04i_irval%02i' %(qval, irval))
print("reading " + the_file)
di = get_dict(the_file)
bt = partfunc(di['vals'][:, mval, :])

qval = 3002
the_file = get_widest_range_file(mtracedir, 'mtrace_qval%04i_irval%02i' %(qval, irval))
print("reading " + the_file)
di = get_dict(the_file)
indt = partfunc(di['vals'][:, mval, :])

qval = 3005
the_file = get_widest_range_file(mtracedir, 'mtrace_qval%04i_irval%02i' %(qval, irval))
print("reading " + the_file)
di = get_dict(the_file)
sheart = partfunc(di['vals'][:, mval, :])

qval = 3008
the_file = get_widest_range_file(mtracedir, 'mtrace_qval%04i_irval%02i' %(qval, irval))
print("reading " + the_file)
di = get_dict(the_file)
advt = partfunc(di['vals'][:, mval, :])

qval = 3011
the_file = get_widest_range_file(mtracedir, 'mtrace_qval%04i_irval%02i' %(qval, irval))
print("reading " + the_file)
di = get_dict(the_file)
compt = partfunc(di['vals'][:, mval, :])

qval = 3
the_file = get_widest_range_file(mtracedir, 'mtrace_qval%04i_irval%02i' %(qval, irval))
print("reading " + the_file)
di = get_dict(the_file)
vp_m0 = partfunc(di['vals'][:, 0, :])

reading /nobackup/lmatilsk/case_M_tspec-lowcad/data/mtrace_mmax010/mtrace_qval0802_irval02-17751000_21286000.pkl
reading /nobackup/lmatilsk/case_M_tspec-lowcad/data/mtrace_mmax010/mtrace_qval3002_irval02-17751000_21286000.pkl
reading /nobackup/lmatilsk/case_M_tspec-lowcad/data/mtrace_mmax010/mtrace_qval3005_irval02-17751000_21286000.pkl
reading /nobackup/lmatilsk/case_M_tspec-lowcad/data/mtrace_mmax010/mtrace_qval3008_irval02-17751000_21286000.pkl
reading /nobackup/lmatilsk/case_M_tspec-lowcad/data/mtrace_mmax010/mtrace_qval3011_irval02-17751000_21286000.pkl
reading /nobackup/lmatilsk/case_M_tspec-lowcad/data/mtrace_mmax010/mtrace_qval0003_irval02-17751000_21286000.pkl


In [19]:
# Let's remove the differential rotation 
times = di['times']
dbtdt =  np.gradient(bt, times, axis=0)
xx_tl = radlevs.radius[irval]*sint.reshape((1, nt))
dbtdt_nodr = dbtdt + 1.0j*vp_m0/x_

In [17]:
np.gradient

<function numpy.gradient(f, *varargs, axis=None, edge_order=1)>